In [1]:
import pandas as pd

In [2]:
DATA_LOCATION = "data"
FILENAME = "attack-effectiveness-"

def get_filename(name):
    return f"{DATA_LOCATION}/{FILENAME}{name}.csv"

get_filename("xcit")

'data/attack-effectiveness-xcit.csv'

In [3]:
def relative_diff(x1, x2):
    return (x2 - x1) / x1

In [4]:
def process_df(model_name):
    filename = get_filename(model_name)
    raw_df = pd.read_csv(filename)
    processed_df = (
        raw_df[["adv_loss", "attack_steps", "epoch"]]
            .drop_duplicates(subset=["attack_steps", "epoch"], ignore_index=True) # Drop mistakenly doubly run attakcs
            .sort_values(by=["epoch", "attack_steps"])
            .reset_index().drop(["index"], axis=1)
            .pivot(index='epoch', columns='attack_steps', values=['adv_loss'])["adv_loss"]
    )
    return processed_df


def compute_relative_differences(df, attack_steps=(1, 2, 10, 100), extra_diffs=((1, 100), (10, 100))):
    rel_diffs_df = pd.DataFrame(index=df.index)
    for s1, s2 in zip(attack_steps, [100] * len(attack_steps)):
        rel_diffs_df[f"{s1}_{s2}"] = relative_diff(df[s1], df[s2])
    # for s1, s2 in extra_diffs:
    #     rel_diffs_df[f"{s1}_{s2}"] = relative_diff(df[s1], df[s2])    
    return rel_diffs_df
                 

def compute_attack_effectiveness(model_name):
    df = process_df(model_name)
    relative_diffs_df = compute_relative_differences(df)
    return relative_diffs_df

xcit_rel_diffs = compute_attack_effectiveness("xcit")
resnet_rel_diffs = compute_attack_effectiveness("resnet")

In [5]:
xcit_rel_diffs

,1_100,2_100,10_100,100_100
epoch,,,,
0,0.125714,0.065132,0.009928,0.0
9,0.023861,0.011308,0.002448,0.0
19,0.028820,0.013642,0.002876,0.0
29,0.033382,0.016928,0.004251,0.0
39,0.035429,0.016857,0.002607,0.0
49,0.044350,0.022382,0.005418,0.0
59,0.045601,0.021767,0.004516,0.0
69,0.051747,0.024708,0.004134,0.0
79,0.062716,0.031544,0.007553,0.0


In [6]:
resnet_rel_diffs

,1_100,2_100,10_100,100_100
epoch,,,,
0,0.010337,0.007342,0.002762,0.0
9,0.038139,0.024182,0.007737,0.0
19,0.045740,0.028730,0.008940,0.0
29,0.047758,0.029835,0.009136,0.0
39,0.078047,0.049098,0.014394,0.0
49,0.084925,0.053367,0.015248,0.0
59,0.097872,0.062603,0.017287,0.0
69,0.119717,0.077093,0.021175,0.0
79,0.134425,0.087580,0.023611,0.0


In [7]:
xcit_rel_diffs < resnet_rel_diffs

,1_100,2_100,10_100,100_100
epoch,,,,
0,False,False,False,False
9,True,True,True,False
19,True,True,True,False
29,True,True,True,False
39,True,True,True,False
49,True,True,True,False
59,True,True,True,False
69,True,True,True,False
79,True,True,True,False


In [8]:
resnet_rel_diffs / xcit_rel_diffs

,1_100,2_100,10_100,100_100
epoch,,,,
0,0.082222,0.112723,0.278202,NaN
9,1.598403,2.138464,3.160829,NaN
19,1.587103,2.106046,3.108477,NaN
29,1.430646,1.762472,2.149029,NaN
39,2.202888,2.912629,5.521363,NaN
49,1.914858,2.384355,2.814290,NaN
59,2.146256,2.876091,3.827705,NaN
69,2.313490,3.120165,5.122327,NaN
79,2.143396,2.776470,3.126201,NaN
